In [ ]:
!pip install -q diffusers

# College project

In [ ]:
!pip install --upgrade -q gradio

In [ ]:
!pip install -q accelerate

In [ ]:

from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
import numpy as np
import torch

import cv2
from PIL import Image


In [ ]:

import PIL

In [ ]:

image = PIL.Image.open('/content/ip1.png')


In [ ]:

image = load_image(
    "/content/ip1.png"
)


In [ ]:
#input image of dimension 512 x 512 , format .png
image

In [ ]:


# load control net and stable diffusion v1-5
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16,
     safety_checker = None,
    requires_safety_checker = False
)

In [ ]:
# speed up diffusion process with faster scheduler and memory optimization

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()



In [ ]:
def get_control_image(image):
  np_image = np.array(image)
  # get canny image
  np_image = cv2.Canny(np_image, 100, 200)
  np_image = np_image[:, :, None]
  np_image = np.concatenate([np_image, np_image, np_image], axis=2)
  canny_image = PIL.Image.fromarray(np_image)
  return canny_image
#canny_image = get_control_image(image)

In [ ]:
#pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
#pipe.enable_model_cpu_offload()
def generate(ip_image,control_image,prompt):
  generator = torch.manual_seed(0)
  image = pipe(
      prompt=prompt,
      num_inference_steps=30,
      generator=generator,
      image=ip_image,
      control_image=control_image,
  ).images[0]
  return image
#generate(image,canny_image,"an abstract art of japanese women")

In [ ]:
# prompt: save the generated image
image.save("gen_girl.jpg")


In [ ]:
# image generation function
def image_to_image(input_png_image, prompt):
  image = PIL.Image.fromarray(input_png_image)
  image = load_image(image)
  control_image = get_control_image(image)
  generated_image = generate(image,control_image,prompt)
  return generated_image

In [ ]:
img  =  image_to_image("/content/ip1.png",'a hyper realistic indian women')

In [ ]:
img

In [ ]:
import gradio as gr

from gradio import Image, Textbox
from gradio import Image


interface = gr.Interface(
    fn = image_to_image,
    inputs=[Image(label="Input Image"), Textbox(label="Text Prompt")],
    outputs=Image(label="Generated Image"),
    title="Text-Driven Visual Translation: Sketch-to-Photo Guided Synthesis",
    description="Use pretrained Stable Diffusion model so as to perform Text-Driven Image Synthesis.",
   elem_id="my-image-to-image-app",
)

# Launch the Gradio interface in the notebook
interface.launch(share=True,debug=True)


In [ ]:
!gradio app.py

In [ ]:
!pip install PIL

In [ ]:
!gradio deploy

In [ ]:
0